<img src="https://files.cdn.thinkific.com/file_uploads/133292/images/d57/1b0/1a9/P03.jpg">

`MCSDSC02P03V3______`

`MCSDSELIMC02P03V2250621`

`MCSDSELIMC02P03V1130621`

<h3><font color="#89DD00">Комментарий ассистента</font></h3>

Проект принят!  
Отличная работа! Комментарии ниже.

## Постановка задачи

Необходимо, обработать данные о страховых, представленные страховой компанией. С помощью полученных данных ответить на следующие вопросы:

1) есть ли зависимость между возрастной группой и статусом выплаты?

2) есть ли зависимость между причиной страхового случая и статусом выплаты?

3) есть ли зависимость между доходом клиента и статусом выплаты?

## Импорт данных и библиотек

Импортируем библиотеки, прочитаем файл `insurance.csv` и получим общую информацию о данных.

In [136]:
import pandas as pd
from nltk.stem import SnowballStemmer
import numpy as np
from pymystem3 import Mystem
russian_stemmer = SnowballStemmer('russian')
m = Mystem()
bo_data=pd.read_csv("../../data/insurance.csv")
bo_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2301 entries, 0 to 2300
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              2301 non-null   int64  
 1   perc_of_compensation_paid       2301 non-null   float64
 2   age_in_days                     2301 non-null   float64
 3   age_id                          2301 non-null   int64  
 4   age_group                       2301 non-null   object 
 5   income                          2236 non-null   float64
 6   count_3-6_months_late           2301 non-null   int64  
 7   count_6-12_months_late          2301 non-null   int64  
 8   count_more_than_12_months_late  2301 non-null   int64  
 9   cause                           2301 non-null   object 
dtypes: float64(3), int64(5), object(2)
memory usage: 179.9+ KB


Столбцы содержат следующую информацию:

* Идентификационный номер
* Процент выплаченной компенсации
* Возраст пользователя в днях
* Идентификационный номер возрастной группы
* Наименование возрастной группы
* Доход пользователя
* Полис оформлен не позднее 3-6 месяцев
* Полис оформлен не позднее 6-12 месяцев
* Полис оформлен больше года назад
* Причина страхового случая

### Выводы

* Данная таблица содержит данные состоящие из 10 граф и 2 301 строки
* Графа "Доход пользователя" имеет пропуски

<h3><font color="#89DD00">Комментарий ассистента</font></h3>

Тут все супер.

## Предобработка данных

**Обработка пропусков**

Проверим данные на наличие пропусков, просуммировав их.

In [137]:
bo_data.isnull().sum()

id                                 0
perc_of_compensation_paid          0
age_in_days                        0
age_id                             0
age_group                          0
income                            65
count_3-6_months_late              0
count_6-12_months_late             0
count_more_than_12_months_late     0
cause                              0
dtype: int64

Заполним пропуски в столбце `income`.

In [138]:
bo_data=bo_data.fillna('0')

Проверяем, что пропусков в `income` не осталось.

In [139]:
bo_data.isnull().sum()

id                                0
perc_of_compensation_paid         0
age_in_days                       0
age_id                            0
age_group                         0
income                            0
count_3-6_months_late             0
count_6-12_months_late            0
count_more_than_12_months_late    0
cause                             0
dtype: int64

**Замена типа данных**

Проверяем, что в столбцах `age_in_days` и `income` вещественный тип данных.

In [140]:
bo_data[['age_in_days','income']].dtypes

age_in_days    float64
income          object
dtype: object

Для этих столбцов нужен целочисленный тип, поэтому переведем их, а также избавимся от отрицательных значений в `income`.

In [141]:
bo_data[['age_in_days','income']]=bo_data[['age_in_days','income']].astype(int)
bo_data[['age_in_days','income']].dtypes

age_in_days    int64
income         int64
dtype: object

In [142]:
bo_data[['income']]=bo_data[['income']].abs()

**Поиск дубликатов с учетом регистра**

Проверим уникальные категории в столбце `age_group`.

In [143]:
bo_data['age_group'].unique()

array(['Пожилой', 'пожилой', 'Зрелый', 'Молодой', 'ЗРЕЛЫЙ', 'ПОЖИЛОЙ',
       'зрелый', 'молодой', 'МОЛОДОЙ'], dtype=object)

Приведем их к нижнему регистру.

In [144]:
bo_data['age_group']=bo_data['age_group'].str.lower()

**Обработка дубликатов**

Установим количество явных дубликатов. Если найдутся, то удалим и снова проверим, что их не осталось.

In [145]:
bo_data.duplicated().sum()

7

In [146]:
bo_data=bo_data.drop_duplicates().reset_index(drop=True)

In [147]:
bo_data.duplicated().sum()

0

<h3><font color="#89DD00">Комментарий ассистента</font></h3>

Тоже без замечаний, всё четко!

**Стэмминг**

Посчитаем уникальные значения в столбце `cause`.

In [148]:
bo_data['cause'].value_counts()

кража автомобиля               112
травма во время отдыха         107
сломала ногу в отпуске          95
упало дерево на автомобиль      94
украли багаж в отпуске          94
травма в отпуске                94
сгорел дом                      93
замкнуло электричество дома     91
сгорела квартира                91
врезались в автомобиль          91
несчастный случай в отпуске     91
сотрясение во время отдыха      90
проникли в жилье                89
повреждение автомобиля          89
потоп в квартире                88
хищение автомобиля (каско)      88
несчастный случай на отдыхе     85
повреждение авто                85
кража авто                      82
обокрали дачу                   81
поломка авто                    81
кража в квартире                79
украли авто                     78
взломали квартиру               77
дтп                             76
затопило дом                    73
Name: cause, dtype: int64

Выделим очевидные разные стеммы: 

 1) отдых, отпуск
 
 2) дтп, авто
 
 3) дом, дача, квартира, жилье

Из них можно выделить следующие виды страхования: 

* Случаи во время отпуска
* Случаи связанные с автомобилями
* Случаи связанные с недвижимым имуществом

Напишем функцию, которая принимает неизменяемую часть слова и возвращает все названия категорий с ним. Сначала попробуем стемминг и если не поймаем «чужих» названий — значит этот метод подойдет.

In [149]:
def stem_cat(stem_list):
    bo_data_stem=bo_data['cause'].drop_duplicates()
    cat_list=[]
    for category in bo_data_stem:
        for word in category.split():
            stem_word = russian_stemmer.stem(word)
            if stem_word==stem_list:
                cat_list.append(category)
    return cat_list

In [150]:
stem_house=('дом')
stem_cat(stem_house)

['сгорел дом', 'замкнуло электричество дома', 'затопило дом']

<h3><font color="#89DD00">Комментарий ассистента</font></h3>

Отлично!

**Категоризация данных**

Cловарь — распространенный способ хранения информации, который помогает убрать текстовые параметры из основной таблицы и увеличить эффективность работы с данными. Выделим словарь для возрастных групп.

In [151]:
age_gr_list=bo_data[['age_id','age_group']].drop_duplicates().sort_values(by = 'age_id')
age_gr_list

,age_id,age_group
4,2,молодой
2,3,зрелый
0,4,пожилой


Категоризируем причины выплаты страховых платежей с помощью лемматизации.

In [152]:
print(bo_data[['cause','id']])

                            cause      id
0          повреждение автомобиля     649
1      упало дерево на автомобиль   81136
2          сломала ногу в отпуске   70762
3      хищение автомобиля (каско)   53935
4     несчастный случай в отпуске   15476
...                           ...     ...
2289                          дтп   16440
2290             кража в квартире  109416
2291             проникли в жилье    4268
2292                обокрали дачу   85708
2293       повреждение автомобиля  105186

[2294 rows x 2 columns]


In [153]:
def lem(row):
        l=m.lemmatize(row)
        print(l)
        if 'авто' in l or 'дтп' in l or 'автомобиль' in l:
            return 'Случаи с автомобилями'
        elif 'отдых' in l or 'отпуск' in l:
            return 'Случаи во время отпуска'
        elif 'дом' in l or 'дача' in l or 'квартира' in l or 'жилье' in l:
            return 'Случаи с недвижимостью'
        
bo_data['cause']=bo_data['cause'].apply(lem)

['повреждение', ' ', 'автомобиль', '\n']
['упасть', ' ', 'дерево', ' ', 'на', ' ', 'автомобиль', '\n']
['сломать', ' ', 'нога', ' ', 'в', ' ', 'отпуск', '\n']
['хищение', ' ', 'автомобиль', ' (', 'каско', ')\n']
['несчастный', ' ', 'случай', ' ', 'в', ' ', 'отпуск', '\n']
['врезаться', ' ', 'в', ' ', 'автомобиль', '\n']
['травма', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['повреждение', ' ', 'автомобиль', '\n']
['сгорать', ' ', 'квартира', '\n']
['сгорать', ' ', 'дом', '\n']
['травма', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['кража', ' ', 'автомобиль', '\n']
['украсть', ' ', 'авто', '\n']
['потоп', ' ', 'в', ' ', 'квартира', '\n']
['украсть', ' ', 'багаж', ' ', 'в', ' ', 'отпуск', '\n']
['травма', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['травма', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['повреждение', ' ', 'авто', '\n']
['проникать', ' ', 'в', ' ', 'жилье', '\n']
['повреждение', ' ', 'автомобиль', '\n']
['украсть', ' ', 'авто', '\n']
['сгорать', ' ', 'дом', '\n']
[

['упасть', ' ', 'дерево', ' ', 'на', ' ', 'автомобиль', '\n']
['повреждение', ' ', 'авто', '\n']
['украсть', ' ', 'авто', '\n']
['кража', ' ', 'в', ' ', 'квартира', '\n']
['обкрадывать', ' ', 'дача', '\n']
['дтп', '\n']
['украсть', ' ', 'багаж', ' ', 'в', ' ', 'отпуск', '\n']
['несчастный', ' ', 'случай', ' ', 'в', ' ', 'отпуск', '\n']
['украсть', ' ', 'багаж', ' ', 'в', ' ', 'отпуск', '\n']
['поломка', ' ', 'авто', '\n']
['потоп', ' ', 'в', ' ', 'квартира', '\n']
['повреждение', ' ', 'авто', '\n']
['кража', ' ', 'авто', '\n']
['обкрадывать', ' ', 'дача', '\n']
['кража', ' ', 'авто', '\n']
['повреждение', ' ', 'авто', '\n']
['упасть', ' ', 'дерево', ' ', 'на', ' ', 'автомобиль', '\n']
['потоп', ' ', 'в', ' ', 'квартира', '\n']
['кража', ' ', 'в', ' ', 'квартира', '\n']
['украсть', ' ', 'багаж', ' ', 'в', ' ', 'отпуск', '\n']
['несчастный', ' ', 'случай', ' ', 'на', ' ', 'отдых', '\n']
['травма', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['потоп', ' ', 'в', ' ', 'квартира', '\n']
['к

['сгорать', ' ', 'дом', '\n']
['несчастный', ' ', 'случай', ' ', 'в', ' ', 'отпуск', '\n']
['повреждение', ' ', 'автомобиль', '\n']
['потоп', ' ', 'в', ' ', 'квартира', '\n']
['кража', ' ', 'в', ' ', 'квартира', '\n']
['замыкать', ' ', 'электричество', ' ', 'дом', '\n']
['дтп', '\n']
['проникать', ' ', 'в', ' ', 'жилье', '\n']
['сотрясение', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['повреждение', ' ', 'авто', '\n']
['взламывать', ' ', 'квартира', '\n']
['сотрясение', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['сотрясение', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['сгорать', ' ', 'квартира', '\n']
['сломать', ' ', 'нога', ' ', 'в', ' ', 'отпуск', '\n']
['упасть', ' ', 'дерево', ' ', 'на', ' ', 'автомобиль', '\n']
['замыкать', ' ', 'электричество', ' ', 'дом', '\n']
['украсть', ' ', 'багаж', ' ', 'в', ' ', 'отпуск', '\n']
['затапливать', ' ', 'дом', '\n']
['замыкать', ' ', 'электричество', ' ', 'дом', '\n']
['дтп', '\n']
['травма', ' ', 'в', ' ', 'отпуск', '\n']
['повреждени

['замыкать', ' ', 'электричество', ' ', 'дом', '\n']
['упасть', ' ', 'дерево', ' ', 'на', ' ', 'автомобиль', '\n']
['украсть', ' ', 'авто', '\n']
['дтп', '\n']
['травма', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['врезаться', ' ', 'в', ' ', 'автомобиль', '\n']
['врезаться', ' ', 'в', ' ', 'автомобиль', '\n']
['кража', ' ', 'в', ' ', 'квартира', '\n']
['замыкать', ' ', 'электричество', ' ', 'дом', '\n']
['украсть', ' ', 'авто', '\n']
['украсть', ' ', 'авто', '\n']
['травма', ' ', 'в', ' ', 'отпуск', '\n']
['сломать', ' ', 'нога', ' ', 'в', ' ', 'отпуск', '\n']
['обкрадывать', ' ', 'дача', '\n']
['хищение', ' ', 'автомобиль', ' (', 'каско', ')\n']
['потоп', ' ', 'в', ' ', 'квартира', '\n']
['сотрясение', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['повреждение', ' ', 'автомобиль', '\n']
['проникать', ' ', 'в', ' ', 'жилье', '\n']
['взламывать', ' ', 'квартира', '\n']
['несчастный', ' ', 'случай', ' ', 'в', ' ', 'отпуск', '\n']
['взламывать', ' ', 'квартира', '\n']
['врезаться', ' '

['затапливать', ' ', 'дом', '\n']
['дтп', '\n']
['кража', ' ', 'в', ' ', 'квартира', '\n']
['проникать', ' ', 'в', ' ', 'жилье', '\n']
['обкрадывать', ' ', 'дача', '\n']
['повреждение', ' ', 'автомобиль', '\n']


Категоризируем статус выплаты компенсации — 0 выплат, выплачена часть, полностью выплачена.

In [154]:
def pay_stat(pay):
    if pay == 1:
        return 'Выплачена'
    if pay == 0:
        return 'Нет выплат'
    else:
        return 'Выплачена часть'
    
bo_data['paided']=bo_data['perc_of_compensation_paid'].apply(pay_stat)

Категоризируем `income`, разбив данные по квантилям.

In [178]:
bo_data['quant']=pd.qcut(bo_data['income'],5)

## Результаты

Подготовим 3 таблицы и изучим результат.

In [184]:
pd.pivot_table(bo_data, index=['age_group'],columns=['paided'],values=['id'],aggfunc=['count'])

count                           
                 id                           
paided    Выплачена Выплачена часть Нет выплат
age_group                                     
зрелый          100             884         62
молодой          52             206         31
пожилой         204             698         57

*ответьте на вопрос — есть ли зависимость между возрастной группой и статусом выплаты?*

Зависимости между возрастной группой и статусом выплаты - нет, но количество зрелых клиентов получают меньше полных выплат, однако, это возмещается тем, что они больше получают не полных выплат.

In [186]:
pd.pivot_table(bo_data, index=['cause'],columns='paided',values='id',aggfunc=['count'])

count                           
paided                  Выплачена Выплачена часть Нет выплат
cause                                                       
Случаи во время отпуска        78             536         42
Случаи с автомобилями         182             640         54
Случаи с недвижимостью         96             612         54

*есть ли зависимость между причиной страхового случая и статусом выплаты?*

Зависимости между причиной страхового случая и статусом выплаты - нет, но по случаям с автомобилями полных выплат больше чем в остальных группах, но это возмещается тем, что у них меньше частичных выплат.

In [187]:
pd.pivot_table(bo_data, index=['quant'],columns='paided',values='id',aggfunc='count')

paided,Выплачена,Выплачена часть,Нет выплат
quant,,,
"(-0.001, 90060.0]",94,326,43
"(90060.0, 135758.0]",69,356,30
"(135758.0, 189668.0]",59,366,33
"(189668.0, 269868.0]",69,371,19
"(269868.0, 2700040.0]",65,369,25


*есть ли зависимость между доходом клиента и статусом выплаты?*

Зависимости между доходом клиента и статусом выплаты - нет, по всем группам разница не более 6%.

<h3><font color="#89DD00">Комментарий ассистента</font></h3>

Замечательная работа. Замечаний нет)